In [2]:
import pretty_midi
import sys
sys.path.append('../')

In [3]:
midi_data = pretty_midi.PrettyMIDI('../data/raw/Part1(PS1)/Bach/Cello Suite 3_BWV1009_2217_cs3-1pre.mid')
midi_data

In [4]:
midi_data = pretty_midi.PrettyMIDI('../data/raw/Part1(PS1)/Beethoven/Piano Sonata No 1 in F minor_OP2NO1_2422_ps01_01.mid')
midi_data

c:\Users\okafo\anaconda3\envs\ocp-models\lib\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [5]:
midi_data = pretty_midi.PrettyMIDI('../data/raw/Part1(PS2)/0.549470161204349_adj.mid')

In [6]:
# extract all possible musical features from the midi file
tempo = midi_data.estimate_tempo()
time_signature = midi_data.time_signature_changes
key_signature = midi_data.key_signature_changes
instruments = midi_data.instruments
notes = midi_data.get_piano_roll(fs=100)
lyrics = midi_data.lyrics
duration = midi_data.get_end_time()
# Create a dictionary to store all the features
midi_features = {
    'tempo': tempo,
    'time_signature': time_signature,
    'key_signature': key_signature,
    'instruments': instruments,
    'notes': notes,
    'lyrics': lyrics,
    'duration': duration
}
midi_features

{'tempo': 166.62404231266297,
 'time_signature': [TimeSignature(numerator=1, denominator=4, time=0.0),
  TimeSignature(numerator=3, denominator=4, time=0.352941)],
 'key_signature': [],
 'instruments': [Instrument(program=0, is_drum=False, name="SFL"),
  Instrument(program=0, is_drum=False, name="SFL")],
 'notes': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'lyrics': [],
 'duration': 337.77556021875}

In [7]:
import music21

# Load the midi file
mf = music21.midi.MidiFile()
mf.open('../data/raw/Part1(PS2)/0.1960551158929671_adj.mid')
mf.read()
mf.close()

In [8]:
# extract all features
notes = []
for element in s.recurse():
    if isinstance(element, music21.note.Note):
        notes.append(element.pitch)
    elif isinstance(element, music21.chord.Chord):
        notes.append('.'.join(str(n) for n in element.pitches))
    elif isinstance(element, music21.note.Rest):
        notes.append('Rest')
notes

NameError: name 's' is not defined

[<music21.stream.Part 0x1db48fb0af0>,
 <music21.stream.Part 0x1db4be42e20>,
 <music21.stream.Part 0x1db504ea850>,
 <music21.stream.Part 0x1db50601ee0>]

In [14]:
# extract features from the music file using music21

# Load the midi file
mf = music21.midi.MidiFile()

# Open the midi file
# mf.open('../data/raw/Part1(PS1)/Beethoven/Cello Sonata No. 3 in A major_OP69_2523_vcs3_3.mid')
# mf.open('../data/raw/Part1(PS1)/Brahms/Horn Trio in E-flat major_OP40_2158_bra40_1.mid')
mf.open('../data/raw/Part1(PS1)/Beethoven/Piano Sonata No 17 in D minor_OP31NO2_2391_ps17_01.mid')

# Read the midi file
mf.read()

# Close the midi file
mf.close()

# Parse the midi file
s = music21.midi.translate.midiFileToStream(mf)

# Show the midi file
s.show('midi')

# extract all features
notes = []
for element in s.recurse():
    if isinstance(element, music21.note.Note):
        notes.append(element.pitch)
    elif isinstance(element, music21.chord.Chord):
        notes.append('.'.join(str(n) for n in element.pitches))
    elif isinstance(element, music21.note.Rest):
        notes.append('Rest')
notes

KeyError: 3042178891584

In [95]:
from music21 import converter, instrument, note, chord, stream, key
import numpy as np
import glob


midi_data = converter.parse('../data/Part1(PS1)/Beethoven/Octet in E-flat major for Winds_OP103_2507_op103-4.mid')
midi_data


# get important features for ML model
key_name_name = midi_data.analyze('key').tonic.name
key_name_mode = midi_data.analyze('key').mode

# Average pitch
pitches = []
for elem in midi_data.flat.notes:
    if isinstance(elem, note.Note):
        pitches.append(elem.pitch.ps)  # MIDI pitch value
    elif isinstance(elem, chord.Chord):
        pitches.extend(p.ps for p in elem.pitches)  # Extract all pitches from the chord

average_pitch = np.mean(pitches) if pitches else 0

# Pitch range
pitch_range = np.ptp(pitches) if pitches else 0  # Peak-to-peak (max-min) range

# Average pitch interval
pitch_intervals = []
for i in range(1, len(pitches)):
    pitch_intervals.append(pitches[i] - pitches[i - 1])
average_pitch_interval = np.mean(pitch_intervals) if pitch_intervals else 0

# Average note duration
durations = [elem.quarterLength for elem in midi_data.flat.notes if isinstance(elem, (note.Note, chord.Chord))]
average_note_duration = np.mean(durations) if durations else 0

# Number of instruments
number_of_instruments = len(midi_data.parts)

# Number of notes
number_of_notes = len([elem for elem in midi_data.flat.notes if isinstance(elem, note.Note)])


# Name of the instruments
instruments = [part.partName for part in midi_data.parts]

# Number of chords
number_of_chords = len([elem for elem in midi_data.flat.notes if isinstance(elem, chord.Chord)])

# Number of time signature changes
number_of_time_signature_changes = len(midi_data.flat.getTimeSignatures())

# Number of key signature changes
key_signatures = [ks for ks in midi_data.recurse() if isinstance(ks, key.KeySignature)]
number_of_key_signature_changes = len(key_signatures)

In [96]:
# create a dictionary to store all the features and dataframes

music_features = {
    'key_name_name': key_name_name,
    'key_name_mode': key_name_mode,
    'average_pitch': average_pitch,
    'pitch_range': pitch_range,
    'average_pitch_interval': average_pitch_interval,
    'average_note_duration': average_note_duration,
    'number_of_instruments': number_of_instruments,
    'number_of_notes': number_of_notes,
    'instruments': instruments,
    'number_of_chords': number_of_chords,
    'number_of_time_signature_changes': number_of_time_signature_changes,
    'number_of_key_signature_changes': number_of_key_signature_changes
}
music_features

{'key_name_name': 'E-',
 'key_name_mode': 'major',
 'average_pitch': 65.1400582472862,
 'pitch_range': 59.0,
 'average_pitch_interval': -0.008209745762711865,
 'average_note_duration': 0.9651840084723325,
 'number_of_instruments': 8,
 'number_of_notes': 3777,
 'instruments': ['Horn',
  'Horn',
  'Oboe',
  'Oboe',
  'Clarinet',
  'Clarinet',
  'Bassoon',
  'Bassoon'],
 'number_of_chords': 0,
 'number_of_time_signature_changes': 8,
 'number_of_key_signature_changes': 0}

In [93]:
# create a dictionary to store all the features and dataframes

music_features = {
    'key_name_name': key_name_name,
    'key_name_mode': key_name_mode,
    'average_pitch': average_pitch,
    'pitch_range': pitch_range,
    'average_pitch_interval': average_pitch_interval,
    'average_note_duration': average_note_duration,
    'number_of_instruments': number_of_instruments,
    'number_of_notes': number_of_notes,
    'instruments': instruments,
    'number_of_chords': number_of_chords,
    'number_of_time_signature_changes': number_of_time_signature_changes,
    'number_of_key_signature_changes': number_of_key_signature_changes
}
music_features

{'key_name_name': 'F',
 'key_name_mode': 'minor',
 'average_pitch': 63.6470054446461,
 'pitch_range': 57.0,
 'average_pitch_interval': 0.0022696323195642307,
 'average_note_duration': 0.824696545284779,
 'number_of_instruments': 2,
 'number_of_notes': 1528,
 'instruments': ['Right', 'Left'],
 'number_of_chords': 257,
 'number_of_time_signature_changes': 4,
 'number_of_key_signature_changes': 2}

In [99]:
from music21 import converter, instrument, note, chord, stream, key, tempo
import numpy as np

# Load MIDI file
midi_data = converter.parse('../data/Part1(PS1)/Beethoven/Violin Sonata No 1 in D major_OP12NO1_2330_lvb12n1a.mid')

# Key features
key_estimation = midi_data.analyze('key')
key_name = key_estimation.tonic.name
key_mode = key_estimation.mode
key_strength = key_estimation.correlationCoefficient  # Key strength

# Pitch features
pitches = []
for elem in midi_data.flat.notes:
    if isinstance(elem, note.Note):
        pitches.append(elem.pitch.ps)  # MIDI pitch
    elif isinstance(elem, chord.Chord):
        pitches.extend(p.ps for p in elem.pitches)

pitch_class_histogram = np.histogram([p % 12 for p in pitches], bins=np.arange(13))[0]  # Pitch class histogram
median_pitch = np.median(pitches) if pitches else 0
std_dev_pitch = np.std(pitches) if pitches else 0
unique_pitch_classes = len(set([int(p) % 12 for p in pitches]))

# Rhythmic features
durations = [elem.quarterLength for elem in midi_data.flat.notes if isinstance(elem, (note.Note, chord.Chord))]
note_density = len(durations) / (midi_data.quarterLength if midi_data.quarterLength else 1)
rhythmic_variance = np.var(durations) if durations else 0
rest_proportion = len([elem for elem in midi_data.flat.notesAndRests if elem.isRest]) / len(durations) if durations else 0

# Harmonic features
chords = [elem for elem in midi_data.flat.notes if isinstance(elem, chord.Chord)]
chord_progressions = [chord.Chord(elem.pitches).commonName for elem in chords]
chord_complexity = sum(1 for chord_obj in chords if len(chord_obj.pitches) > 3) / len(chords) if chords else 0

# Instrumental features
instrument_types = [part.partName for part in midi_data.parts]
instrument_count = len(midi_data.parts)

# Tempo and structural features
tempo_changes = midi_data.flat.getElementsByClass(tempo.MetronomeMark)
average_tempo = np.mean([t.number for t in tempo_changes]) if tempo_changes else 0
time_signature_changes = len(midi_data.flat.getTimeSignatures())
measure_count = len(midi_data.getElementsByClass(stream.Measure))

# Create dictionary of features
features = {
    "key_name": key_name,
    "key_mode": key_mode,
    "key_strength": key_strength,
    "average_pitch": np.mean(pitches) if pitches else 0,
    "median_pitch": median_pitch,
    "std_dev_pitch": std_dev_pitch,
    "pitch_range": np.ptp(pitches) if pitches else 0,
    "unique_pitch_classes": unique_pitch_classes,
    "note_density": note_density,
    "rhythmic_variance": rhythmic_variance,
    "rest_proportion": rest_proportion,
    "chord_complexity": chord_complexity,
    "instrument_count": instrument_count,
    "instrument_types": instrument_types,
    "average_tempo": average_tempo,
    "time_signature_changes": time_signature_changes,
    "measure_count": measure_count,
    "chord_progressions": chord_progressions,
}

# Print features
features


{'key_name': 'D',
 'key_mode': 'major',
 'key_strength': 0.8989814867441954,
 'average_pitch': 65.30672332675671,
 'median_pitch': 66.0,
 'std_dev_pitch': 12.930820472783276,
 'pitch_range': 64.0,
 'unique_pitch_classes': 12,
 'note_density': 3.581104154809334,
 'rhythmic_variance': 0.3071941703970694,
 'rest_proportion': 0.44712862894681077,
 'chord_complexity': 0.07071428571428572,
 'instrument_count': 2,
 'instrument_types': ['Violin', 'Piano'],
 'average_tempo': 126.69642943305186,
 'time_signature_changes': 6,
 'measure_count': 0,
 'chord_progressions': ['major triad',
  'Perfect Octave',
  'Perfect Octave',
  'Perfect Octave',
  'Perfect Octave',
  'Perfect Octave',
  'Perfect Octave',
  'Perfect Octave',
  'Perfect Octave',
  'Perfect Octave',
  'Perfect Octave',
  'Perfect Octave',
  'Perfect Octave',
  'Perfect Octave',
  'Perfect Octave',
  'Perfect Octave',
  'Perfect Octave',
  'Perfect Octave',
  'Perfect Fifth',
  'major triad',
  'major triad',
  'major triad',
  'major 

In [116]:
from music21 import converter, note, chord, key, tempo
import numpy as np
from collections import Counter

# Load MIDI file
midi_data = converter.parse('../data/Part1(PS1)/Beethoven/Cello Sonata No. 3 in A major_OP69_2523_vcs3_3.mid')

# Initialize feature extraction
# -------------------------------------------------

# 1. Key Features
key_estimation = midi_data.analyze('key')
key_name = key_estimation.tonic.name  # E.g., "C", "D"
key_mode = key_estimation.mode        # E.g., "major", "minor"
key_strength = key_estimation.correlationCoefficient  # Confidence in key estimation

# 2. Pitch Features
pitches = []
for elem in midi_data.flat.notes:
    if isinstance(elem, note.Note):
        pitches.append(elem.pitch.ps)  # MIDI pitch value
    elif isinstance(elem, chord.Chord):
        pitches.extend(p.ps for p in elem.pitches)  # Add pitches from all notes in the chord

average_pitch = np.mean(pitches) if pitches else 0
median_pitch = np.median(pitches) if pitches else 0
std_dev_pitch = np.std(pitches) if pitches else 0
pitch_range = np.ptp(pitches) if pitches else 0  # Peak-to-peak (max - min) pitch
unique_pitch_classes = len(set(int(p) % 12 for p in pitches))  # Unique pitch classes (C, C#, ..., B)

# 3. Rhythmic Features
durations = [elem.quarterLength for elem in midi_data.flat.notes if isinstance(elem, (note.Note, chord.Chord))]
note_density = len(durations) / (midi_data.quarterLength if midi_data.quarterLength else 1)  # Notes per measure
rhythmic_variance = np.var(durations) if durations else 0  # Variance in note durations
rest_proportion = len([elem for elem in midi_data.flat.notesAndRests if elem.isRest]) / len(durations) if durations else 0

# 4. Harmonic Features
chords = [elem for elem in midi_data.flat.notes if isinstance(elem, chord.Chord)]
chord_progressions = [chord.Chord(elem.pitches).commonName for elem in chords]
chord_diversity = len(set(chord_progressions)) if chord_progressions else 0  # Number of unique chords
most_common_chord = Counter(chord_progressions).most_common(1)[0][0] if chord_progressions else None
consonant_chords = {'major triad', 'minor triad', 'perfect fifth', 'major seventh'}
num_consonant_chords = sum(1 for chord in chord_progressions if chord in consonant_chords)
consonance_ratio = num_consonant_chords / len(chord_progressions) if chord_progressions else 0

# 5. Instrument Features
instruments = [part.partName for part in midi_data.parts if part.partName]
num_instruments = len(set(instruments))  # Number of unique instruments
instrument_diversity = len(set(instruments)) if instruments else 0
most_common_instrument = max(set(instruments), key=instruments.count) if instruments else None

# 6. Tempo and Structural Features
tempo_changes = midi_data.flat.getElementsByClass(tempo.MetronomeMark)
average_tempo = np.mean([t.number for t in tempo_changes]) if tempo_changes else 0
time_signature_changes = len(midi_data.flat.getTimeSignatures())
measure_count = len(midi_data.getElementsByClass('Measure'))

# Create a dictionary of extracted features
# -------------------------------------------------
features = {
    "key_name": key_name,
    "key_mode": key_mode,
    "key_strength": key_strength,
    "average_pitch": average_pitch,
    "median_pitch": median_pitch,
    "std_dev_pitch": std_dev_pitch,
    "pitch_range": pitch_range,
    "unique_pitch_classes": unique_pitch_classes,
    "note_density": note_density,
    "rhythmic_variance": rhythmic_variance,
    "rest_proportion": rest_proportion,
    "chord_diversity": chord_diversity,
    "most_common_chord": most_common_chord,
    "consonance_ratio": consonance_ratio,
    "num_instruments": num_instruments,
    "instrument_diversity": instrument_diversity,
    "most_common_instrument": most_common_instrument,
    "average_tempo": average_tempo,
    "time_signature_changes": time_signature_changes,
    "measure_count": measure_count,
}

# Print the feature dictionary
features


{'key_name': 'A',
 'key_mode': 'major',
 'key_strength': 0.9174018336464617,
 'average_pitch': 62.48881292728403,
 'median_pitch': 62.0,
 'std_dev_pitch': 12.931539117049956,
 'pitch_range': 65.0,
 'unique_pitch_classes': 12,
 'note_density': 4.670036764705882,
 'rhythmic_variance': 0.2586550370386917,
 'rest_proportion': 0.16355048218854557,
 'chord_diversity': 31,
 'most_common_chord': 'Perfect Octave',
 'consonance_ratio': 0.16095534787123572,
 'num_instruments': 3,
 'instrument_diversity': 3,
 'most_common_instrument': 'Cello',
 'average_tempo': 141.11904761904762,
 'time_signature_changes': 6,
 'measure_count': 0}

In [124]:
from music21 import converter, note, chord, key, tempo
import numpy as np
from collections import Counter

# Load MIDI file
midi_data = converter.parse('../data/raw/Part1(PS1)/Bach/Cello Suite 3_BWV1009_2217_cs3-1pre.mid')

# ----------------------------
# 1. Key Features
# ----------------------------
key_estimation = midi_data.analyze('key')
key_name = key_estimation.tonic.name  # Tonic (e.g., "C", "D")
key_mode = key_estimation.mode        # Major or minor mode
key_strength = key_estimation.correlationCoefficient  # Confidence in key estimation

# Count key signature changes and summarize the timeline
key_signatures = [ks for ks in midi_data.recurse() if isinstance(ks, key.KeySignature)]
num_key_signature_changes = len(key_signatures)
most_frequent_key_signature = (
    Counter([ks.sharps for ks in key_signatures]).most_common(1)[0][0] if key_signatures else None
)

# ----------------------------
# 2. Pitch Features
# ----------------------------
pitches = []
for elem in midi_data.flat.notes:
    if isinstance(elem, note.Note):
        pitches.append(elem.pitch.ps)
    elif isinstance(elem, chord.Chord):
        pitches.extend(p.ps for p in elem.pitches)

average_pitch = np.mean(pitches) if pitches else 0
median_pitch = np.median(pitches) if pitches else 0
std_dev_pitch = np.std(pitches) if pitches else 0
pitch_range = np.ptp(pitches) if pitches else 0
unique_pitch_classes = len(set(int(p) % 12 for p in pitches))

# Replace pitch class distribution with entropy (a scalar representation)
pitch_class_distribution = Counter(int(p) % 12 for p in pitches)
pitch_entropy = -sum(
    (freq / len(pitches)) * np.log2(freq / len(pitches))
    for freq in pitch_class_distribution.values()
) if pitches else 0

# Extract melodic intervals and summarize
melodic_intervals = [pitches[i+1] - pitches[i] for i in range(len(pitches) - 1)]
average_melodic_interval = np.mean(melodic_intervals) if melodic_intervals else 0

# ----------------------------
# 3. Rhythmic Features
# ----------------------------
durations = [elem.quarterLength for elem in midi_data.flat.notes if isinstance(elem, (note.Note, chord.Chord))]
note_density = len(durations) / (midi_data.quarterLength if midi_data.quarterLength else 1)
rhythmic_variance = np.var(durations) if durations else 0
rest_proportion = len([elem for elem in midi_data.flat.notesAndRests if elem.isRest]) / len(durations) if durations else 0

# ----------------------------
# 4. Harmonic Features
# ----------------------------
chords = [elem for elem in midi_data.flat.notes if isinstance(elem, chord.Chord)]
chord_progressions = [chord.Chord(elem.pitches).commonName for elem in chords]
chord_diversity = len(set(chord_progressions)) if chord_progressions else 0
most_common_chord = Counter(chord_progressions).most_common(1)[0][0] if chord_progressions else None

# Compute consonance ratio
consonant_chords = {'major triad', 'minor triad', 'perfect fifth', 'major seventh'}
num_consonant_chords = sum(1 for chord in chord_progressions if chord in consonant_chords)
consonance_ratio = num_consonant_chords / len(chord_progressions) if chord_progressions else 0

# ----------------------------
# 5. Instrument Features
# ----------------------------
instruments = [part.partName for part in midi_data.parts if part.partName]
num_instruments = len(set(instruments))
instrument_diversity = len(set(instruments)) if instruments else 0
most_common_instrument = max(set(instruments), key=instruments.count) if instruments else None

# ----------------------------
# 6. Tempo and Structural Features
# ----------------------------
tempo_changes = midi_data.flat.getElementsByClass(tempo.MetronomeMark)
tempo_values = [t.number for t in tempo_changes] if tempo_changes else []
average_tempo = np.mean(tempo_values) if tempo_values else 0
min_tempo = min(tempo_values) if tempo_values else 0
max_tempo = max(tempo_values) if tempo_values else 0
tempo_variability = np.std(tempo_values) if tempo_values else 0

# Count time signature changes and summarize
time_signatures = [ts.ratioString for ts in midi_data.flat.getTimeSignatures()]
time_signature_changes = len(time_signatures)
most_frequent_time_signature = (
    Counter(time_signatures).most_common(1)[0][0] if time_signatures else None
)

# Structural details
measure_count = len(midi_data.getElementsByClass('Measure'))
total_duration = midi_data.duration.quarterLength

# ----------------------------
# 7. Compile All Features
# ----------------------------
features = {
    "key_name": key_name,
    "key_mode": key_mode,
    "key_strength": key_strength,
    "num_key_signature_changes": num_key_signature_changes,
    "most_frequent_key_signature": most_frequent_key_signature,
    "average_pitch": average_pitch,
    "median_pitch": median_pitch,
    "std_dev_pitch": std_dev_pitch,
    "pitch_range": pitch_range,
    "unique_pitch_classes": unique_pitch_classes,
    "pitch_entropy": pitch_entropy,
    "average_melodic_interval": average_melodic_interval,
    "note_density": note_density,
    "rhythmic_variance": rhythmic_variance,
    "rest_proportion": rest_proportion,
    "chord_diversity": chord_diversity,
    "most_common_chord": most_common_chord,
    "consonance_ratio": consonance_ratio,
    "num_instruments": num_instruments,
    "instrument_diversity": instrument_diversity,
    "most_common_instrument": most_common_instrument,
    "average_tempo": average_tempo,
    "min_tempo": min_tempo,
    "max_tempo": max_tempo,
    "tempo_variability": tempo_variability,
    "time_signature_changes": time_signature_changes,
    "most_frequent_time_signature": most_frequent_time_signature,
    "measure_count": measure_count,
    "total_duration": total_duration,
}

# Print the feature dictionary
features


{'key_name': 'C',
 'key_mode': 'major',
 'key_strength': 0.8837165527149484,
 'num_key_signature_changes': 4,
 'most_frequent_key_signature': 0,
 'average_pitch': 52.871896722939425,
 'median_pitch': 53.0,
 'std_dev_pitch': 6.164130042672554,
 'pitch_range': 31.0,
 'unique_pitch_classes': 12,
 'pitch_entropy': 3.1341504253700063,
 'average_melodic_interval': -0.016898608349900597,
 'note_density': 3.8143939393939394,
 'rhythmic_variance': 0.04402055521971818,
 'rest_proportion': 0.26216484607745777,
 'chord_diversity': 0,
 'most_common_chord': None,
 'consonance_ratio': 0,
 'num_instruments': 4,
 'instrument_diversity': 4,
 'most_common_instrument': '--------------------------------------',
 'average_tempo': 52.5,
 'min_tempo': 35.0,
 'max_tempo': 70.0,
 'tempo_variability': 17.5,
 'time_signature_changes': 4,
 'most_frequent_time_signature': '3/4',
 'measure_count': 0,
 'total_duration': 264.0}

In [125]:
instruments

['Solo Cello',
 '--------------------------------------',
 'Johann Sebastian Bach  (1685-1750)',
 'Six Suites for Solo Cello']

In [126]:
import pandas as pd
features_df = pd.DataFrame(features, index=[0])
features_df

,key_name,key_mode,key_strength,num_key_signature_changes,most_frequent_key_signature,average_pitch,median_pitch,std_dev_pitch,pitch_range,unique_pitch_classes,...,instrument_diversity,most_common_instrument,average_tempo,min_tempo,max_tempo,tempo_variability,time_signature_changes,most_frequent_time_signature,measure_count,total_duration
0,C,major,0.883717,4,0,52.871897,53.0,6.16413,31.0,12,...,4,--------------------------------------,52.5,35.0,70.0,17.5,4,3/4,0,264.0


In [10]:
import pandas as pd
df = pd.read_csv('../data/processed/processed_data_unknown.csv')
df

,key_name,key_mode,key_strength,num_key_signature_changes,most_frequent_key_signature,average_pitch,median_pitch,std_dev_pitch,pitch_range,unique_pitch_classes,...,most_common_instrument,average_tempo,min_tempo,max_tempo,tempo_variability,time_signature_changes,most_frequent_time_signature,measure_count,total_duration,composer
0,C,major,0.931962,4,0.0,65.218633,66.0,10.545467,55.0,12,...,SFL,102.750000,40.0,130.0,28.340563,8,3/4,0,963.0,unknown
1,B,minor,0.908940,3,2.0,68.676913,71.0,11.108826,51.0,12,...,SFL,100.000000,88.0,108.0,6.825558,3,3/4,0,375.0,unknown
2,E-,major,0.887868,4,-3.0,66.488552,67.0,11.192664,68.0,12,...,SFL,130.403846,104.0,144.0,10.986460,8,1/4,0,1112.0,unknown
3,C,minor,0.861038,2,-3.0,58.044706,58.0,11.411310,53.0,12,...,SFL,275.333333,200.0,300.0,24.729649,2,3/4,0,648.0,unknown
4,C,minor,0.920158,0,NaN,58.891206,59.0,7.498501,44.0,12,...,NaN,69.554688,22.0,110.0,25.224617,2,4/4,0,156.0,unknown
5,A,minor,0.934357,1,0.0,76.657807,76.0,5.783972,30.0,11,...,SFL,70.000000,70.0,70.0,0.000000,1,3/4,0,138.0,unknown
6,C,minor,0.859349,3,-3.0,53.251613,53.0,5.644973,31.0,12,...,SFL,190.000000,130.0,250.0,60.000000,3,3/4,0,387.0,unknown
7,G,major,0.873430,6,1.0,66.677091,66.0,9.638568,45.0,12,...,SFL,107.892857,60.0,120.0,13.283554,4,1/4,0,364.0,unknown
8,C,major,0.815727,4,1.0,63.665437,64.0,12.945813,60.0,12,...,SFL,262.895833,200.0,360.0,24.832556,4,2/2,0,1652.0,unknown
9,A,minor,0.958729,1,0.0,74.868687,75.0,5.322974,29.0,12,...,SFL,133.333333,50.0,250.0,84.983659,1,3/4,0,255.0,unknown


In [11]:
# check for missing values
df.isnull().sum()

key_name                        1
key_mode                        1
key_strength                    0
num_key_signature_changes       0
most_frequent_key_signature     5
average_pitch                   0
median_pitch                    0
std_dev_pitch                   0
pitch_range                     0
unique_pitch_classes            0
pitch_entropy                   0
average_melodic_interval        0
note_density                    0
rhythmic_variance               0
rest_proportion                 0
chord_diversity                 0
most_common_chord               4
consonance_ratio                0
num_instruments                 0
instrument_diversity            0
most_common_instrument          4
average_tempo                   0
min_tempo                       0
max_tempo                       0
tempo_variability               0
time_signature_changes          0
most_frequent_time_signature    0
measure_count                   0
total_duration                  0
composer      

In [12]:
# rows with missing values
df[df.isnull().any(axis=1)]

,key_name,key_mode,key_strength,num_key_signature_changes,most_frequent_key_signature,average_pitch,median_pitch,std_dev_pitch,pitch_range,unique_pitch_classes,...,most_common_instrument,average_tempo,min_tempo,max_tempo,tempo_variability,time_signature_changes,most_frequent_time_signature,measure_count,total_duration,composer
4,C,minor,0.920158,0,NaN,58.891206,59.0,7.498501,44.0,12,...,NaN,69.554688,22.0,110.0,25.224617,2,4/4,0,156.0,unknown
5,A,minor,0.934357,1,0.0,76.657807,76.0,5.783972,30.0,11,...,SFL,70.000000,70.0,70.0,0.000000,1,3/4,0,138.0,unknown
6,C,minor,0.859349,3,-3.0,53.251613,53.0,5.644973,31.0,12,...,SFL,190.000000,130.0,250.0,60.000000,3,3/4,0,387.0,unknown
10,E,minor,0.930421,0,NaN,62.279173,62.0,14.588394,73.0,12,...,Piano,125.400000,107.0,160.0,18.413039,3,4/4,0,792.5,unknown
21,G#,minor,0.926383,0,NaN,61.406250,61.5,10.116932,47.0,12,...,NaN,59.214286,35.0,70.0,9.748103,2,6/8,0,90.0,unknown
23,E-,major,0.908816,6,-3.0,65.067314,65.0,9.736874,45.0,11,...,SFL,192.961538,159.5,216.0,16.745219,6,3/4,0,462.0,unknown
24,C#,minor,0.956537,0,NaN,63.897135,66.0,10.109741,48.0,12,...,NaN,98.833333,70.0,114.0,14.769526,4,6/4,0,234.0,unknown
27,C,minor,0.909470,4,-3.0,53.500942,53.0,5.592696,31.0,12,...,SFL,108.285714,62.0,250.0,58.944803,4,4/4,0,324.0,unknown
28,NaN,NaN,0.000000,2,-4.0,61.310316,61.0,10.929180,63.0,12,...,SFL,95.000000,70.0,120.0,25.000000,2,2/4,0,2023/12,unknown
30,C,minor,0.912680,0,NaN,62.396846,65.0,10.492107,48.0,12,...,NaN,67.478333,54.0,80.0,8.755037,3,4/4,0,132.0,unknown
